<a href="https://colab.research.google.com/github/Pathin220/4105_ML_Hw5/blob/main/4105_Hw_5_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#2.a. Develop preprocessing and a training loop to train a linear regression model that predicts housing price based on the following input variables:
#area, bedrooms, bathrooms, stories, parking

#For this, you need to use the housing dataset. For training and validation, use 80% (training) and 20% (validation) split.
# Identify the best parameters for your linear regression model based on the above input variables. In this case, you will have six parameters:

#2.b Use 5000 epochs for your training. Explore different learning rates from 0.1 to 0.0001 (you need four separate trainings).
# Report your loss and validation accuracy for every 500 epochs per training. Pick the best linear model.

#2.c. Compare your results against the linear regression done in homework 1. Do you see meaningful differences?

In [2]:
import torch
import tensorflow
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Machine Learning/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#mapping the variables to a binary output
varlist_1 =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist_1] = housing[varlist_1].apply(mapping)


In [5]:
#Splitting the Dataset
X_train, X_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)


In [6]:
#Splitting Variables
Y_train = X_train.pop("price")
Y_val = X_val.pop("price")

In [7]:
#Standardizing the Dataset
standard = StandardScaler()
vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

X_train_st = X_train
X_train_st[vars] = standard.fit_transform(X_train_st[vars])

X_val_st = X_val
X_val_st[vars] = standard.fit_transform(X_val_st[vars])

In [8]:
#defining the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [9]:
#defining the model
def model_h(t_u, w5, w4, w3, w2, w1, b):
  return w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2 * t_u ** 2 + w1 * t_u + b

In [10]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [11]:
X_train_st_numpy = np.c_[np.ones((len(Y_train), 1)), X_train_st[vars]]
X_val_st_numpy = np.c_[np.ones((len(Y_val), 1)), X_val_st[vars]]

In [12]:
X_train_t = torch.tensor(X_train_st_numpy)
X_val_t = torch.tensor(X_val_st_numpy)
Y_train_t = torch.tensor(Y_train.values)
Y_val_t = torch.tensor(Y_val.values)

In [13]:
Y_train_m = Y_train_t.float().mean()
Y_train_st = Y_train.std()
Y_train_st_t = (Y_train_t-Y_train_m) / Y_train_st

In [14]:
#defining a training loop
def training_loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:
       params.grad.zero_()

    t_p_train = model_h(t_u_train, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

    t_p_val = model_h(t_u_val, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()


    if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params



In [15]:
#SGD 1
SGD_1_learning_rate = 1e-10
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD2 = optim.SGD([params], lr= SGD_1_learning_rate)
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD2,
    params = params,
    t_u_train = X_train_t,
    t_c_train = Y_train_t,
    t_u_val = X_val_t,
    t_c_val = Y_val_t,
)



Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469775109834.8945, Validation loss: 25188976396646.2383

Epoch 3, Training loss: 26469773180553.0859, Validation loss: 25188973784338.6445

Epoch 500, Training loss: 26468815393815.4961, Validation loss: 25187676811293.6172

Epoch 1000, Training loss: 26467853967709.3555, Validation loss: 25186374708878.4023

Epoch 1500, Training loss: 26466894685737.6445, Validation loss: 25185075308681.8281

Epoch 2000, Training loss: 26465937542263.7617, Validation loss: 25183778604007.8789

Epoch 2500, Training loss: 26464982533492.0234, Validation loss: 25182484590249.2070

Epoch 3000, Training loss: 26464029653790.8789, Validation loss: 25181193260458.2383

Epoch 3500, Training loss: 26463078901346.4336, Validation loss: 25179904612739.5859

Epoch 4000, Training loss: 26462130262689.1289, Validation loss: 25178618630664.7695

Epoch 4500, Training loss: 26461183740208.0781, Validation loss:

tensor([65.5474, 23.2831,  8.2987,  6.3373,  2.4985,  4.7773],
       requires_grad=True)

In [16]:
#SGD 2
SGD_2_learning_rate = 1e-15
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
SGD2_optimizer = optim.SGD([params], lr=SGD_2_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = SGD2_optimizer,
    params = params,
    t_u_train = X_train_t,
    t_c_train = Y_train_t,
    t_u_val = X_val_t,
    t_c_val = Y_val_t,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469777039105.4453, Validation loss: 25188979008937.4609

Epoch 3, Training loss: 26469777039089.8711, Validation loss: 25188979008916.1055

Epoch 500, Training loss: 26469777031350.7656, Validation loss: 25188978998304.1562

Epoch 1000, Training loss: 26469777023564.9492, Validation loss: 25188978987628.1445

Epoch 1500, Training loss: 26469777015779.1289, Validation loss: 25188978976952.1328

Epoch 2000, Training loss: 26469777007993.3125, Validation loss: 25188978966276.1289

Epoch 2500, Training loss: 26469777000207.4883, Validation loss: 25188978955600.1211

Epoch 3000, Training loss: 26469776992421.6758, Validation loss: 25188978944924.1094

Epoch 3500, Training loss: 26469776984635.8516, Validation loss: 25188978934248.0977

Epoch 4000, Training loss: 26469776976850.0234, Validation loss: 25188978923572.0781

Epoch 4500, Training loss: 26469776969064.1953, Validation loss:

tensor([1.0006e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 4.7779e-05],
       requires_grad=True)

In [17]:
#SGD 3
SGD_3_learning_rate = 1e-20
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
SGD3_optimizer = optim.SGD([params], lr=SGD_3_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = SGD3_optimizer,
    params = params,
    t_u_train = X_train_t,
    t_c_train = Y_train_t,
    t_u_val = X_val_t,
    t_c_val = Y_val_t,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 3, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8125

Epoch 1000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 1500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 2000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 2500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 3000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 3500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 4000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 4500, Training loss: 26469777039121.0117, Validation loss:

tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 4.7778e-10],
       requires_grad=True)

In [18]:
#SGD 4
SGD_4_learning_rate = 1e-25
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
SGD4_optimizer = optim.SGD([params], lr=SGD_3_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = SGD4_optimizer,
    params = params,
    t_u_train = X_train_t,
    t_c_train = Y_train_t,
    t_u_val = X_val_t,
    t_c_val = Y_val_t,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 3, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8125

Epoch 1000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 1500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 2000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 2500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 3000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 3500, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 4000, Training loss: 26469777039121.0117, Validation loss: 25188979008958.8086

Epoch 4500, Training loss: 26469777039121.0117, Validation loss:

tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 4.7778e-10],
       requires_grad=True)

In [19]:
#Adam Number 1 learning rate of 0.1
Adam1_learning_rate = 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
Adam1_optimizer = optim.Adam([params], lr=Adam1_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = Adam1_optimizer,
    params = params,
    t_u_train = X_train_t,
    t_c_train = X_train_t,
    t_u_val = X_val_t,
    t_c_val = X_val_t,
)

RuntimeError: ignored